In [1]:
# import internal files
import emulators
import kernels
import data
import historymatch
import plot

import numpy as np
import matplotlib.pyplot as plt
import random
from matplotlib.patches import Ellipse

from scipy.interpolate import griddata

plt.rcParams.update({'font.size': 10})

np.random.seed(4)

from scipy import stats
from scipy.stats import norm, uniform
import scipy.optimize
import importlib

In [2]:
def model_eqn_1(x, theta_0, theta_1, theta_2):
    return theta_0 + theta_1*(x) + theta_2*(x**2)

#def model_eqn_1(x, theta_0, theta_1):
    #return theta_0 + theta_1*(x)
    
function1 = np.vectorize(model_eqn_1)

In [3]:
def model_test1(theta_0, theta_1, theta_2):
    return theta_0 + theta_1*(0.1) + theta_2*(0.1**2)

def model_test2(theta_0, theta_1, theta_2):
    return theta_0 + theta_1*(0.2) + theta_2*(0.2**2)

def model_test3(theta_0, theta_1, theta_2):
    return theta_0 + theta_1*(0.3) + theta_2*(0.3**2)

def model_test4(theta_0, theta_1, theta_2):
    return theta_0 - theta_1*np.sin(0.2) + theta_2*np.cos(0.2)

#def model_eqn_1(x, theta_0, theta_1):
    #return theta_0 + theta_1*(x)
    
function1 = np.vectorize(model_eqn_1)

In [4]:
#def model_eqn_2(x, theta_0, theta_1):
    #return theta_0 - theta_1*np.sin(x)

def model_eqn_2(x, theta_0, theta_1, theta_2):
    return theta_0 - theta_1*np.sin(x) + theta_2*np.cos(x)

function2 = np.vectorize(model_eqn_2)

In [5]:
# define parameter space
x_bound = np.array([0.1, 1])
Nx = 4
xvals = np.linspace(x_bound[0], x_bound[-1], Nx)

theta_0_bound = np.array([-1, 1]).reshape(1,-1)
theta_1_bound = np.array([-1, 1]).reshape(1,-1)
theta_2_bound = np.array([-1, 1]).reshape(1,-1)

parameter_bounds = np.concatenate((theta_0_bound, theta_1_bound, theta_2_bound), axis=0)

#parameter_bounds = np.concatenate((theta_0_bound, theta_1_bound), axis=0)

theta_0_vals = np.linspace(parameter_bounds[0,0], parameter_bounds[0,1], 100)
theta_1_vals = np.linspace(parameter_bounds[1,0], parameter_bounds[1,1], 100)
theta_2_vals = np.linspace(parameter_bounds[2,0], parameter_bounds[2,1], 100)


theta_vals = np.concatenate((theta_0_vals.reshape(1,-1), theta_1_vals.reshape(1,-1), theta_2_vals.reshape(1,-1)), axis=0)

#theta_vals = np.concatenate((theta_0_vals.reshape(1,-1), theta_1_vals.reshape(1,-1)), axis=0)

In [6]:
# for testing: true datapoints
true_parameters = [-0.1, 0.3, 0.6]
ndim = len(true_parameters)

var_exp_f1 = 0.001        # observational uncertainty variance
var_exp_f2 = 0.001

error_f1 = np.random.normal(0, var_exp_f1, len(xvals))
error_f2 = np.random.normal(0, var_exp_f2, len(xvals))

z_1 = function1(xvals, true_parameters[0], true_parameters[1], true_parameters[2]) + error_f1 # observed datapoint

z_2 = function2(xvals, true_parameters[0], true_parameters[1], true_parameters[2]) + error_f2  # observed datapoint


z1test = model_test1(true_parameters[0], true_parameters[1], true_parameters[2]) + np.random.normal(0, var_exp_f1) # observed datapoint
z2test = model_test2(true_parameters[0], true_parameters[1], true_parameters[2]) + np.random.normal(0, var_exp_f1)  # observed datapoint
z3test = model_test3(true_parameters[0], true_parameters[1], true_parameters[2]) + np.random.normal(0, var_exp_f1)  # observed datapoint
z4test = model_test4(true_parameters[0], true_parameters[1], true_parameters[2]) + np.random.normal(0, var_exp_f1)

In [7]:
#true_model = [model_eqn_1, model_eqn_2]
true_model = [model_test1, model_test2, model_test3, model_test4]
obs_data = np.array([z1test, z2test, z3test, z4test])

#obs_data = np.concatenate((z_1.reshape(1,-1),z_2.reshape(1,-1)), axis=0)

In [8]:
nwaves = 1
ndim = 3

In [9]:


HM = historymatch.HistoryMatch(obs_data,
                               ndim, true_model, 'GP', 'hypercube',
                               parameter_bounds, var_exp_f1, 0)

results = HM.run(nwaves=1)

Running wave 1


NameError: name 'locate_boundaries' is not defined

In [ ]:
importlib.reload(plot)

fig, axes = plt.subplots(ndim,ndim,figsize=(10,10))
#plot.opticaldepth_1D(results.samples[0], parameter_bounds[0], parameter = 0, bins=20, Fig=(fig,ax1))
    
#plot.plot_implausibility2D(results.samples[0], parameter_bounds, [0,1], bins=20)

plot.plotcorner(results.samples[0], parameter_bounds, 3, Fig=(fig,axes), labels=('theta1', 'theta2', 'theta3'))